In [1]:
import requests
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
apikey='GAVwTDFxk3gZFwVwgnpppDp85ORYwB0z'

# List of symbols you want to fetch data for
symbols = ['AAPL']

# Initialize an empty DataFrame to hold all data
df = pd.DataFrame()

for symbol in symbols:
    # Modify the URL or parameters to fetch data for the current symbol
    url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={apikey}'
    
    # Making a GET request to the API for the current symbol
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Assuming the data for each symbol is under a 'historical' key
        # This might need to be adjusted based on the actual structure of the API response
        hdf = pd.DataFrame(data['historical'])
        
        # Add a column for the symbol
        hdf['symbol'] = symbol
        
        # Concatenate the current symbol's data with the overall dataset
        df = pd.concat([df, hdf], ignore_index=True)
    else:
        print(f'Failed to fetch data for {symbol}:', response.status_code)


In [3]:
df['date'] = pd.to_datetime(df['date'])

In [9]:


def get_log_boxsize(percent):
    """
    Determine an appropriate boxsize for logarithmic scale P&F analysis.

    Parameters:
    percent (float or np.array): The percent change for which to calculate the box size.

    Returns:
    float or np.array: The calculated box size on a logarithmic scale.
    """
    if not np.isscalar(percent) and not isinstance(percent, np.ndarray):
        raise ValueError("Argument percent must be a numeric value or a numpy array")
    
    percent = np.array(percent)  # Ensure percent is an array for vectorized operations
    if np.any(percent <= -100):
        print("Warning: Percent contains values less than or equal to -100, introducing NaNs and strange results!")
    elif np.any(percent <= 0):
        print("Warning: Percent contains values less than or equal to zero, introducing strange results!")

    return np.log(100 + percent) - np.log(100)

def quote_to_scale(x, log):
    return np.log(x) if log else x

def quote_to_boxnumber(quote, status, boxsize, log):
    scaled = quote_to_scale(quote, log)
    return (np.floor(scaled / boxsize) if status == "X"
            else np.ceil(scaled / boxsize))

def scale_to_quote(boxnumber, boxsize, log):
    scale = boxnumber * boxsize
    return np.exp(scale) if log else scale

def next_box(quote, status, boxsize=1, log=False):
    boxnum = quote_to_boxnumber(quote, status, boxsize, log)
    return scale_to_quote(boxnum + (1 if status == "X" else -1),
                          boxsize, log)

def next_reversal(quote, status, reversal, boxsize, log):
    boxnum = quote_to_boxnumber(quote, status, boxsize, log)
    return scale_to_quote(boxnum - reversal if status == "X" else
                          boxnum + reversal, boxsize, log)

def xo_processor(df, reversal=3, boxsize=1, log=False):
    df['status_xo'] = np.nan
    df['boxnumber'] = np.nan
    df['status_bs'] = np.nan
    df['nextX'] = np.nan
    df['lastNextX'] = np.nan
    df['nextO'] = np.nan
    df['lastNextO'] = np.nan
    df['column'] = np.nan

    # Initialize first row
    df.at[0, 'status_xo'] = 'X'
    df.at[0, 'boxnumber'] = quote_to_boxnumber(df.at[0, 'high'], 'X', boxsize, log)
    df.at[0, 'status_bs'] = 'Buy'
    df.at[0, 'nextX'] = next_box(df.at[0, 'high'], 'X', boxsize, log)
    df.at[0, 'lastNextX'] = df.at[0, 'nextX']
    df.at[0, 'nextO'] = next_box(df.at[0, 'low'], 'O', boxsize, log)
    df.at[0, 'lastNextO'] = df.at[0, 'nextO']
    df.at[0, 'column'] = 1

    for i in range(1, len(df)):
        prev = df.iloc[i-1]
        curr_high = df.at[i, 'high']
        curr_low = df.at[i, 'low']
        status = prev['status_xo']

        if status == 'X':
            if curr_high >= prev['nextX']:
                update_for_x(df, i, curr_high, boxsize, log, reversal)
            elif curr_low <= prev['nextO']:
                update_for_o(df, i, curr_low, boxsize, log, reversal, prev)
            else:
                copy_prev_row(df, i, prev)
        elif status == 'O':
            if curr_low <= prev['nextO']:
                update_for_o(df, i, curr_low, boxsize, log, reversal)
            elif curr_high >= prev['nextX']:
                update_for_x(df, i, curr_high, boxsize, log, reversal, prev)
            else:
                copy_prev_row(df, i, prev)

    return df

def update_for_x(df, i, quote, boxsize, log, reversal, prev=None):
    df.at[i, 'status_xo'] = 'X'
    df.at[i, 'boxnumber'] = quote_to_boxnumber(quote, 'X', boxsize, log)
    df.at[i, 'nextX'] = next_box(quote, 'X', boxsize, log)
    df.at[i, 'nextO'] = (next_reversal(quote, 'X', reversal, boxsize, log)
                         if prev is None else prev['nextO'])
    df.at[i, 'column'] = (df.at[i-1, 'column'] + 1
                          if prev is not None and prev['status_xo'] == 'O'
                          else df.at[i-1, 'column'])
    df.at[i, 'status_bs'] = 'Buy'

def update_for_o(df, i, quote, boxsize, log, reversal, prev=None):
    df.at[i, 'status_xo'] = 'O'
    df.at[i, 'boxnumber'] = quote_to_boxnumber(quote, 'O', boxsize, log)
    df.at[i, 'nextO'] = next_box(quote, 'O', boxsize, log)
    df.at[i, 'nextX'] = (next_reversal(quote, 'O', reversal, boxsize, log)
                         if prev is None else prev['nextX'])
    df.at[i, 'column'] = (df.at[i-1, 'column'] + 1
                          if prev is not None and prev['status_xo'] == 'X'
                          else df.at[i-1, 'column'])
    df.at[i, 'status_bs'] = 'Sell'

def copy_prev_row(df, i, prev):
    df.iloc[i, 2:] = prev[2:]


In [5]:
df = xo_processor(df)

In [6]:
df


,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,...,changeOverTime,symbol,status_xo,boxnumber,status_bs,nextX,lastNextX,nextO,lastNextO,column
0,2024-02-02,179.860,187.33,179.25,185.85,185.85,101096412,100565746,5.99,3.33000,...,0.033300,AAPL,X,187.0,Buy,188.0,188.0,179.0,179.0,1.0
1,2024-02-01,183.985,187.33,179.25,185.85,185.85,101096412,100565746,5.99,3.33000,...,0.033300,AAPL,X,187.0,Buy,188.0,188.0,179.0,179.0,1.0
2,2024-01-31,187.040,187.33,179.25,185.85,185.85,101096412,100565746,5.99,3.33000,...,0.033300,AAPL,X,187.0,Buy,188.0,188.0,179.0,179.0,1.0
3,2024-01-30,190.940,191.80,187.47,188.04,188.04,55836970,55859400,-2.90,-1.52000,...,-0.015200,AAPL,X,191.0,Buy,192.0,NaN,188.0,NaN,1.0
4,2024-01-29,192.010,192.20,189.58,191.73,191.73,47145622,47145600,-0.28,-0.14583,...,-0.001458,AAPL,X,192.0,Buy,193.0,NaN,189.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,2019-02-11,42.760,43.27,42.38,43.23,41.67,95997600,95997600,0.65,1.53000,...,0.015300,AAPL,O,43.0,Sell,46.0,NaN,42.0,NaN,248.0
1254,2019-02-08,42.250,43.27,42.38,43.23,41.67,95997600,95997600,0.65,1.53000,...,0.015300,AAPL,O,43.0,Sell,46.0,NaN,42.0,NaN,248.0
1255,2019-02-07,43.100,43.27,42.38,43.23,41.67,95997600,95997600,0.65,1.53000,...,0.015300,AAPL,O,43.0,Sell,46.0,NaN,42.0,NaN,248.0
1256,2019-02-06,43.660,43.27,42.38,43.23,41.67,95997600,95997600,0.65,1.53000,...,0.015300,AAPL,O,43.0,Sell,46.0,NaN,42.0,NaN,248.0


In [7]:
import matplotlib.pyplot as plt

def plot_xo_chart(df):
    fig, ax = plt.subplots(figsize=(10, 6))

    # Iterate through the DataFrame to plot Xs and Os
    for i, row in df.iterrows():
        if row['status_xo'] == 'X':
            ax.text(i, row['boxnumber'], 'X', color='green', fontsize=12, ha='center')
        elif row['status_xo'] == 'O':
            ax.text(i, row['boxnumber'], 'O', color='red', fontsize=12, ha='center')

    # Setting the plot aesthetics
    ax.set_xlabel('Index')
    ax.set_ylabel('Box Number')
    plt.title('Point & Figure Chart of X and O')
    plt.grid(True)
    plt.xticks(rotation=45)

    plt.show()


In [8]:
plot_xo_chart(df)

ValueError: Image size of 974252x92026 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1000x600 with 1 Axes>